In [2]:
import pandas as pd
import seaborn as sns
sns.set() 
%matplotlib inline 
pd.set_option('max_columns',999)

In [ ]:
# PAYMENT
pay_data = pd.read_csv("data//PANEL//PAYMENT data.csv", engine = "python")
pay_brand = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_cat = pd.read_csv("data//PANEL//PAYMENT Summary_Brand.csv", engine = "python")
pay_total = pd.read_csv("data//PANEL//PAYMENT Summary_Total.csv", engine = "python")

In [ ]:
# PPDB
ppdb_data = pd.read_csv("data//PANEL//PPDB data.csv", engine = "python")

In [ ]:
# APP
app_time = pd.read_csv("data//PANEL//APP Refine_AppTime.csv", engine = "python")
app_cat_g = pd.read_csv("data//PANEL//APP Refine_CategoryTime_G.csv", engine = "python")
app_cat_mme = pd.read_csv("data//PANEL//APP Refine_CategoryTime_MME.csv", engine = "python")
app_sum_time = pd.read_csv("data//PANEL//APP Refine_Usage_Sum_Time.csv", engine = "python")

# AR

### 카톡 설치여부 reverse (해볼 것)

누구나 다 1인 사건(ex 카톡 설치여부)은 정보량이 적은 사건인데, 이것을 카톡 미설치여부로 바꾸면 1인사람이 거의 없어서 정보량이 많아짐

이렇게하면 두 사건의 동시발생확률이 0.5 이상이 되는것을 줄일 수 있음 그 lift값의 2배라는 기준때문에 사라지는 것을 줄일 수 있음.

--> 전체 샘플의 반 이상이 1인 사건은 반 이하가 1인 사건으로 바꾸기


In [4]:
# 세 데이터 합치기

pay_dummies = pd.read_csv("data/0109_payment_dummies.csv").drop("Unnamed: 0", axis =1)
ps_dummies = pd.read_csv("data/0109_ps_dummies.csv").drop("Unnamed: 0", axis =1)
app_dummies = pd.read_csv("data/0109_app_dummies.csv").drop("Unnamed: 0", axis =1)

al = pd.merge(pay_dummies, ps_dummies, on = "PANEL_ID")
al = pd.merge(al, app_dummies, on = "PANEL_ID")

# panel_id 제거
del al["PANEL_ID"]

In [5]:
# al 길이는 10591
al.shape

(10591, 779)

In [6]:
al_2 = pd.DataFrame(al.sum()).reset_index()
al_2.columns = ["label", "sum"]

In [7]:
al_2.head(3)

,label,sum
0,PAY_APPROVAL_TYPE_L,10586
1,PAY_APPROVAL_TYPE_F,1009
2,PAY_CATEGORY_CODE__1101,3981


In [9]:
al_2[al_2["sum"] > 5295][:50].sort_values(by=['sum'], ascending = False).head(10)

,label,sum
65,PAY_SMS_REGISTRATION_MONTH_11,10591
0,PAY_APPROVAL_TYPE_L,10586
3,PAY_CATEGORY_CODE__1799,10014
101,PAY_SMS_REGISTRATION_TIME__18,8639
102,PAY_SMS_REGISTRATION_TIME__12,8378
100,PAY_SMS_REGISTRATION_TIME__17,8354
105,PAY_SMS_REGISTRATION_TIME__13,8331
109,PAY_SMS_REGISTRATION_TIME__19,8269
106,PAY_SMS_REGISTRATION_TIME__14,8266
99,PAY_SMS_REGISTRATION_TIME__15,8247


### AR 돌리기

In [10]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [11]:
al.shape

(10591, 779)

In [ ]:
# min_support 0.01 이하는 발생하지 않은 사건으로 취급
# confidence는 일단 고려하지 않음
# lift의 min_threshold를 2로 설정 (두 사건이 독립일때 기대되는 동시발생빈도보다 2배 높은 동시발생빈도)

frequent_itemsets = apriori(al, min_support=0.01, use_colnames=True, max_len=2)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=2)

In [ ]:
# 총 6534개의 규칙 발견, 조건절과 결과절이 서로 뒤바뀐 경우라면 같은 경우로 취급하기 때문에 이보다 규칙 수는 더 적음

len(rules)

In [ ]:
# 중복 규칙을 삭제하는 과정

dup_rule_df = rules.sort_values(by='lift', ascending=False)[['antecedents','consequents']]
dup_rule_df['antecedents'] = dup_rule_df['antecedents'].map(lambda x: str(x).replace('frozenset({','').replace('})','')[1:-1])
dup_rule_df['consequents'] = dup_rule_df['consequents'].map(lambda x: str(x).replace('frozenset({','').replace('})','')[1:-1])

rule_set = []

for i in range(len(dup_rule_df)):
    x = set(dup_rule_df.loc[i,:])
    rule_set.append(x)

rules['dup?'] = rule_set
rules['dup?'] = rules['dup?'].map(lambda x: str(x))
rules = rules.drop_duplicates(subset=['dup?']).sort_values(by='lift', ascending=False)

del rules['dup?']

### AR 결과

lift 최소값 2 기준으로 2958개의 규칙 발견


In [ ]:
# 가능한 룰의 최대 갯수

(779*778)/2

In [ ]:
# 총 3531개의 규칙 발견

len(rules)

In [ ]:
# lift 값이 매우 높은 규칙들은 상식상 당연한 결과

rules.head(10)

In [ ]:
rules_for_network = rules[['antecedents','consequents','lift']]
rules_for_network.head()

In [ ]:
rules_for_network['antecedents'] = rules_for_network['antecedents'].map(lambda x: str(x).replace('frozenset({','').replace('})','')[1:-1])
rules_for_network['consequents'] = rules_for_network['consequents'].map(lambda x: str(x).replace('frozenset({','').replace('})','')[1:-1])
forMatrix = rules_for_network.copy()

In [ ]:
rules_for_network.columns = ['Source','Target', 'Weight']
rules_for_network.head()

In [ ]:
# 분석에 사용할 lift 값만 따로 edgelist형태로 저장

rules_for_network.to_csv('data/AR_allcol_edgelist.csv', index=False)

In [141]:
# # 혹시 몰라서 매트릭스 형태로도 저장

# AR_adj_matrix = pd.DataFrame(index=al.columns, columns=al.columns)

# for i,j,k in zip(forMatrix['antecedents'], forMatrix['consequents'], forMatrix['lift']) :
#     AR_adj_matrix.loc[i,j] = k
#     AR_adj_matrix.loc[j,i] = k   
    
# AR_adj_matrix_fillzero = AR_adj_matrix.fillna(int(0))
# AR_adj_matrix_fillzero.to_csv("data/AR_adj_matrix_fillzero.csv")